In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Prose/Mathieu/Benter-Project

/home/mathieu/Prose/Mathieu/Benter-Project


In [2]:
%matplotlib inline

import datetime as dt
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import re
from itertools import combinations
import tensorflow as tf
import functools
from scipy.stats import rankdata
import scipy
import json

from utils import import_data
from winning_validation import errors
from winning_validation import r_squared
from winning_horse_models import sklearn
from winning_horse_models.dl_shared_layers import LogisticRegressionModel, DLSharedLayersModel
from winning_horse_models.xgboost import XGBoostWinningModel
from winning_horse_models.lgbm import LGBMWinningModel
from winning_horse_models.tpot import TPOTWinningModel
from training_procedures import sequential_training, flattened_training
from constants import Sources, SplitSets
from utils import preprocess

from database.setup import create_sqlalchemy_session
from models.race import Race
from models.runner import Runner
tqdm.pandas()

/home/mathieu/.pyenv/versions/3.7.9/envs/benter-project_venv/lib/python3.7/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/mathieu/.pyenv/versions/3.7.9/envs/benter-project_venv/lib/python3.7/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
SOURCE = Sources.UNIBET
N_FEATURES = preprocess.get_n_preprocessed_feature_columns(source=SOURCE)

In [ ]:
%%capture --no-stdout
%%time
tpot_winning_model, training_history = flattened_training.train_per_n_horses_races(source=SOURCE, winning_model=TPOTWinningModel(source=SOURCE, n_features=N_FEATURES), verbose=True)

No training and validation data for 2


In [ ]:
%%capture --no-stdout
%%time
# Only on 10 n_horses
tpot_winning_model, training_history = flattened_training.train_on_n_horses_races(n_horses=10, source=SOURCE, winning_model=TPOTWinningModel(source=SOURCE, n_features=N_FEATURES, hyperparameters={'verbosity':4, 'n_jobs':-1, 'log_file':'./logs/tpot.logs'}), verbose=True)

32 operators have been imported by TPOT.


In [4]:
%%capture --no-stdout
%%time
layers=tf.keras.Sequential(layers=[tf.keras.layers.Dense(24),tf.keras.layers.Dense(12), tf.keras.layers.Dense(1)], name="shared_layers")
dl_shared_model = DLSharedLayersModel(source=SOURCE, n_features=N_FEATURES, name='24_12', shared_layers=layers)
dl_shared_model, training_history =sequential_training.train_on_each_horse_with_epochs(source=SOURCE, winning_model=dl_shared_model, n_epochs=10,n_epochs_per_n_horses=100, verbose=False)

dl_shared_model.save_model(prefix="48_features_10_epochs_100_epochs_per_n_horse")

1/1 [==============================] - 0s 13ms/step - loss: 4.3640 - categorical_accuracy: 0.0000e+00 - categorical_crossentropy: 4.3640
INFO:tensorflow:Assets written to: ./saved_models/DLSharedLayersModel/48_features_10_epochs_100_epochs_per_n_horse24_12_model/assets
CPU times: user 6h 38min 5s, sys: 29min 23s, total: 7h 7min 29s
Wall time: 4h 8min 44s


In [5]:
%%capture --no-stdout
r_squared.compute_mcfadden_r_squared(source=SOURCE,winning_model=dl_shared_model, verbose=True)

Comparing on same races w/ 3 horses with odds 3 races (4 races in total)
Mean Predicted probas of actual race result: 0.003% (Random: 19.251%, Odds: 40.856%)

Comparing on same races w/ 4 horses with odds 40 races (45 races in total)
Mean Predicted probas of actual race result: 16.272% (Random: 21.939%, Odds: 37.649%)

Comparing on same races w/ 5 horses with odds 287 races (328 races in total)
Mean Predicted probas of actual race result: 11.838% (Random: 19.517%, Odds: 34.718%)

Comparing on same races w/ 6 horses with odds 737 races (900 races in total)
Mean Predicted probas of actual race result: 10.557% (Random: 16.428%, Odds: 27.936%)

Comparing on same races w/ 7 horses with odds 1285 races (1642 races in total)
Mean Predicted probas of actual race result: 7.263% (Random: 14.496%, Odds: 26.404%)

Comparing on same races w/ 8 horses with odds 1764 races (2443 races in total)
Mean Predicted probas of actual race result: 5.994% (Random: 12.687%, Odds: 24.742%)

Comparing on same rac

KeyError: 'predicted_probabilities'

In [4]:
%%capture --no-stdout
%%time
sequential_sgd_regression = LogisticRegressionModel(source=SOURCE, n_features=N_FEATURES)
sequential_sgd_regression, training_history =sequential_training.train_on_each_horse_with_epochs(source=SOURCE, winning_model=sequential_sgd_regression, n_epochs=1, verbose=True)

sequential_sgd_regression.save_model(prefix="48_col_")

Epoch 0

No val data for 2
1/1 [==============================] - 0s 348ms/step - loss: 1.0270 - categorical_accuracy: 0.0000e+00 - categorical_crossentropy: 1.0270
Training for 2 horses (2 races, val 0 races): loss per horse: 0.513, val loss per horse: nan Train Accuracy: 0.0%, Val Accuracy: nan%

1/1 [==============================] - 1s 867ms/step - loss: 1.3859 - categorical_accuracy: 0.3333 - categorical_crossentropy: 1.3859 - val_loss: 1.1093 - val_categorical_accuracy: 0.2500 - val_categorical_crossentropy: 1.1093
Training for 3 horses (27 races, val 4 races): loss per horse: 0.462, val loss per horse: 0.370 Train Accuracy: 33.3%, Val Accuracy: 25.0%

11/11 [==============================] - 1s 26ms/step - loss: 1.3204 - categorical_accuracy: 0.3934 - categorical_crossentropy: 1.3204 - val_loss: 1.5097 - val_categorical_accuracy: 0.3111 - val_categorical_crossentropy: 1.5097
Training for 4 horses (322 races, val 45 races): loss per horse: 0.329, val loss per horse: 0.377 Train A

In [5]:
sequential_sgd_regression = LogisticRegressionModel.load_model(prefix="48_col_")

In [6]:
%%capture --no-stdout
exact_top_1 = errors.compute_validation_error(source=SOURCE, k=1,winning_model=sequential_sgd_regression, validation_method=errors.exact_top_k)
print(errors.compute_overall_average(exact_top_1))

kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=sequential_sgd_regression, 
                                                            validation_method=errors.kappa_cohen_like)

(0.2229023589131084, 0.0924753657808301, 0.32284263959390863)


In [7]:
print(errors.compute_overall_average(kappa_cohen_1))

(0.14700073613270376, 0.003320176199196662, 0.2571769690533504)


In [8]:
r_squared.compute_mcfadden_r_squared(source=SOURCE,winning_model=sequential_sgd_regression, verbose=True)['combined_model_r_squared']

Comparing on same races w/ 3 horses with odds 3 races (4 races in total)
Mean Predicted probas of actual race result: 48.073% (Random: 19.251%, Odds: 40.856%)

Comparing on same races w/ 4 horses with odds 40 races (45 races in total)
Mean Predicted probas of actual race result: 29.962% (Random: 21.939%, Odds: 37.649%)

Comparing on same races w/ 5 horses with odds 287 races (328 races in total)
Mean Predicted probas of actual race result: 27.293% (Random: 19.517%, Odds: 34.718%)

Comparing on same races w/ 6 horses with odds 737 races (900 races in total)
Mean Predicted probas of actual race result: 21.060% (Random: 16.428%, Odds: 27.936%)

Comparing on same races w/ 7 horses with odds 1285 races (1642 races in total)
Mean Predicted probas of actual race result: 19.659% (Random: 14.496%, Odds: 26.404%)

Comparing on same races w/ 8 horses with odds 1764 races (2443 races in total)
Mean Predicted probas of actual race result: 17.482% (Random: 12.687%, Odds: 24.742%)

Comparing on same 

/home/mathieu/Prose/Mathieu/Benter-Project/winning_validation/r_squared.py:163: RuntimeWarning: Mean of empty slice.
  f"Mean Predicted probas of actual race result: "
/home/mathieu/.pyenv/versions/3.7.9/envs/benter-project_venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Comparing on same races w/ 27 horses with odds 0 races (2 races in total)
Mean Predicted probas of actual race result: nan% (Random: nan%, Odds: nan%)

Comparing on same races w/ 28 horses with odds 1 races (10 races in total)
Mean Predicted probas of actual race result: 3.331% (Random: 5.774%, Odds: 3.105%)

Comparing on same races w/ 29 horses with odds 0 races (1 races in total)
Mean Predicted probas of actual race result: nan% (Random: nan%, Odds: nan%)

Comparing on same races w/ 30 horses with odds 0 races (3 races in total)
Mean Predicted probas of actual race result: nan% (Random: nan%, Odds: nan%)

Comparing on same races w/ 31 horses with odds 1 races (2 races in total)
Mean Predicted probas of actual race result: 2.201% (Random: 3.741%, Odds: 2.156%)

Comparing on same races w/ 33 horses with odds 0 races (4 races in total)
Mean Predicted probas of actual race result: nan% (Random: nan%, Odds: nan%)

Comparing on same races w/ 35 horses with odds 0 races (1 races in total)
M

/home/mathieu/Prose/Mathieu/Benter-Project/winning_validation/r_squared.py:200: RuntimeWarning: invalid value encountered in double_scalars
  -n_races * np.log(n_horses)
/home/mathieu/Prose/Mathieu/Benter-Project/winning_validation/r_squared.py:204: RuntimeWarning: invalid value encountered in double_scalars
  odds_r_squared = 1 - np.sum(np.log(odds_proba)) / (-n_races * np.log(n_horses))
/home/mathieu/Prose/Mathieu/Benter-Project/winning_validation/r_squared.py:208: RuntimeWarning: invalid value encountered in double_scalars
  -n_races * np.log(n_horses)


{'n_horses_r_squared': {3: {'model_r_squared': 0.3199518679669403,
   'odds_r_squared': 0.16174488735854864,
   'random_r_squared': -1.1567870085819605,
   'n_races': 3,
   'n_rejected_races': 0},
  4: {'model_r_squared': 0.04838334496364094,
   'odds_r_squared': 0.17443008029053897,
   'random_r_squared': -0.2638522211438925,
   'n_races': 40,
   'n_rejected_races': 0},
  5: {'model_r_squared': 0.1188531971478467,
   'odds_r_squared': 0.24501564971518353,
   'random_r_squared': -0.19893567369267884,
   'n_races': 287,
   'n_rejected_races': 0},
  6: {'model_r_squared': 0.06972517015540647,
   'odds_r_squared': 0.1640584706648457,
   'random_r_squared': -0.15517132130257028,
   'n_races': 737,
   'n_rejected_races': 0},
  7: {'model_r_squared': 0.09557863728221849,
   'odds_r_squared': 0.1870765948715638,
   'random_r_squared': -0.13839741292274388,
   'n_races': 1285,
   'n_rejected_races': 0},
  8: {'model_r_squared': 0.0972932252097275,
   'odds_r_squared': 0.2072795225420433,
   'r

In [9]:
%%capture --no-stdout
%%time
sequential_sgd_regression = LogisticRegressionModel(source=SOURCE, n_features=N_FEATURES)
sequential_sgd_regression, training_history =sequential_training.train_on_each_horse_with_epochs(source=SOURCE, winning_model=sequential_sgd_regression, n_epochs=5, verbose=True)

sequential_sgd_regression.save_model(prefix="48_col_5_epochs_")

Epoch 0

No val data for 2
1/1 [==============================] - 0s 271ms/step - loss: 1.2207 - categorical_accuracy: 0.0000e+00 - categorical_crossentropy: 1.2207
Training for 2 horses (2 races, val 0 races): loss per horse: 0.610, val loss per horse: nan Train Accuracy: 0.0%, Val Accuracy: nan%

1/1 [==============================] - 0s 481ms/step - loss: 1.3419 - categorical_accuracy: 0.2593 - categorical_crossentropy: 1.3419 - val_loss: 0.9882 - val_categorical_accuracy: 0.5000 - val_categorical_crossentropy: 0.9882
Training for 3 horses (27 races, val 4 races): loss per horse: 0.447, val loss per horse: 0.329 Train Accuracy: 25.9%, Val Accuracy: 50.0%

11/11 [==============================] - 0s 21ms/step - loss: 1.3230 - categorical_accuracy: 0.3744 - categorical_crossentropy: 1.3230 - val_loss: 1.5115 - val_categorical_accuracy: 0.3333 - val_categorical_crossentropy: 1.5115
Training for 4 horses (322 races, val 45 races): loss per horse: 0.331, val loss per horse: 0.378 Train A

In [10]:
%%capture --no-stdout
exact_top_1 = errors.compute_validation_error(source=SOURCE, k=1,winning_model=sequential_sgd_regression, validation_method=errors.exact_top_k)
print(errors.compute_overall_average(exact_top_1))

kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=sequential_sgd_regression, 
                                                            validation_method=errors.kappa_cohen_like)

(0.2264257987458943, 0.0924753657808301, 0.32284263959390863)


In [11]:
print(errors.compute_overall_average(kappa_cohen_1))

(0.15088790848989717, 0.003320176199196662, 0.2571769690533504)


In [12]:
r_squared.compute_mcfadden_r_squared(source=SOURCE,winning_model=sequential_sgd_regression, verbose=True)

Comparing on same races w/ 3 horses with odds 3 races (4 races in total)
Mean Predicted probas of actual race result: 49.138% (Random: 19.251%, Odds: 40.856%)

Comparing on same races w/ 4 horses with odds 40 races (45 races in total)
Mean Predicted probas of actual race result: 29.930% (Random: 21.939%, Odds: 37.649%)

Comparing on same races w/ 5 horses with odds 287 races (328 races in total)
Mean Predicted probas of actual race result: 27.692% (Random: 19.517%, Odds: 34.718%)

Comparing on same races w/ 6 horses with odds 737 races (900 races in total)
Mean Predicted probas of actual race result: 21.423% (Random: 16.428%, Odds: 27.936%)

Comparing on same races w/ 7 horses with odds 1285 races (1642 races in total)
Mean Predicted probas of actual race result: 20.109% (Random: 14.496%, Odds: 26.404%)

Comparing on same races w/ 8 horses with odds 1764 races (2443 races in total)
Mean Predicted probas of actual race result: 17.859% (Random: 12.687%, Odds: 24.742%)

Comparing on same 

{'n_horses_r_squared': {3: {'model_r_squared': 0.33145758754856647,
   'odds_r_squared': 0.16174488735854864,
   'random_r_squared': -1.1567870085819605,
   'n_races': 3,
   'n_rejected_races': 0},
  4: {'model_r_squared': 0.033495435272820284,
   'odds_r_squared': 0.17443008029053897,
   'random_r_squared': -0.2638522211438925,
   'n_races': 40,
   'n_rejected_races': 0},
  5: {'model_r_squared': 0.11510322718679211,
   'odds_r_squared': 0.24501564971518353,
   'random_r_squared': -0.19893567369267884,
   'n_races': 287,
   'n_rejected_races': 0},
  6: {'model_r_squared': 0.07255207162942268,
   'odds_r_squared': 0.1640584706648457,
   'random_r_squared': -0.15517132130257028,
   'n_races': 737,
   'n_rejected_races': 0},
  7: {'model_r_squared': 0.09951489200238761,
   'odds_r_squared': 0.1870765948715638,
   'random_r_squared': -0.13839741292274388,
   'n_races': 1285,
   'n_rejected_races': 0},
  8: {'model_r_squared': 0.10015771698589016,
   'odds_r_squared': 0.2072795225420433,
 

In [ ]:
# Pretrain

In [ ]:
%%capture --no-stdout
pretrained_sequential_sgd_regression = LogisticRegressionModel(source=SOURCE,n_features=N_FEATURES)
pretrained_sequential_sgd_regression, training_history =sequential_training.pretrain_on_each_subraces(source=SOURCE,
                                                                                                      winning_model=pretrained_sequential_sgd_regression, 
                                                                                                      n_permutations=10, 
                                                                                                      verbose=True)


exact_top_1 = errors.compute_validation_error(source=SOURCE, k=1,winning_model=pretrained_sequential_sgd_regression, validation_method=errors.exact_top_k)
print(errors.compute_overall_average(exact_top_1))

kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=pretrained_sequential_sgd_regression, 
                                                            validation_method=errors.kappa_cohen_like)
print(errors.compute_overall_average(kappa_cohen_1))

In [ ]:
%%capture --no-stdout

pretrained_sequential_sgd_regression, training_history =sequential_training.train_on_each_horse_with_epochs(source=SOURCE, winning_model=pretrained_sequential_sgd_regression, n_epochs=1, verbose=True)

In [ ]:
%%capture --no-stdout
exact_top_1 = errors.compute_validation_error(source=SOURCE, k=1,winning_model=pretrained_sequential_sgd_regression, validation_method=errors.exact_top_k)
print(errors.compute_overall_average(exact_top_1))

kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=pretrained_sequential_sgd_regression, 
                                                            validation_method=errors.kappa_cohen_like)
print(errors.compute_overall_average(kappa_cohen_1))

In [ ]:
pretrained_sequential_sgd_regression.save_model(prefix="pretrained_")

In [ ]:
# XGBoost

In [4]:
%%time
%%capture --no-stdout

xgboost_winning_model, training_history = flattened_training.train_per_n_horses_races(source=SOURCE, winning_model=XGBoostWinningModel(source=SOURCE), verbose=True)
xgboost_winning_model.save_model(prefix="48_col_")

No training and validation data for 2
[07:16:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training for 3 horses (27 races): loss per horse: 0.019, val loss per horse: nan Train Accuracy: 100.0%, Val Accuracy: nan%

[07:16:41] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Training for 4 horses (322 races): loss per horse: 0.003, val loss per horse: 0.472 Train Accuracy: 100.0%, Val Accuracy: 28.9%

[07:16:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval

TypeError: save_model() got an unexpected keyword argument 'prefix'

CPU times: user 9h 15min 31s, sys: 1min 23s, total: 9h 16min 54s
Wall time: 1h 20min 57s


In [18]:
exact_top_1 = errors.compute_validation_error(source=SOURCE, k=1,winning_model=xgboost_winning_model, validation_method=errors.exact_top_k)
print(errors.compute_overall_average(exact_top_1))

kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=xgboost_winning_model, 
                                                            validation_method=errors.kappa_cohen_like)
print(errors.compute_overall_average(kappa_cohen_1))

IndexError: index 15 is out of bounds for axis 0 with size 15

In [ ]:
r_squared.compute_mcfadden_r_squared(source=SOURCE,winning_model=xgboost_winning_model, verbose=True)

In [ ]:
%%time
for SklearnModel in [
     #sklearn.DecisionTreeModel, sklearn.SVCModel, #sklearn.KNNModel,
                     #sklearn.RandomForestModel, 
    #sklearn.AdaBoostModel,
                     # sklearn.GradientBoostingModel, sklearn.GaussianNBModel, sklearn.LDAModel,
                     sklearn.SGDModel
                    ]:
    print(SklearnModel.__name__)
    sklearn_winning_model, training_history = flattened_training.train_per_n_horses_races(source=SOURCE, winning_model=SklearnModel(), verbose=True)
    sklearn_winning_model.save_model()
    print()

In [ ]:
%%time
sklearn_winning_model, training_history = flattened_training.train_per_n_horses_races(source=SOURCE, winning_model=sklearn.LogisticRegressionModel(), verbose=True)

In [ ]:
sklearn_winning_model.save_model()

In [ ]:
# APPENDIX

In [ ]:
source, on_split=SOURCE, "train"

In [ ]:
n_horses=10
y_format = "first_position"

In [ ]:
rh_df = import_data.get_split_date(source=source, on_split=on_split)

x = []
y = []
race_dfs = []
for _, race_df in rh_df[rh_df["n_horses"] == n_horses].groupby("race_id"):
    break

In [ ]:
import_data.extract_x_y(
            race_df=race_df,
            x_format="sequential_per_horse",
            y_format=y_format,
            source=source,
        )

In [ ]:
y_race = (race_df["horse_place"] == 1).values

In [ ]:
y_race.value

In [ ]:
x_race = preprocess.preprocess(race_horse_df=race_df, source=source)

In [ ]:
pd.isnull(x_race).all().any()

In [ ]:
race_df["horse_place"].isna().all() 

In [ ]:
y_race = (race_df["horse_place"] == 1).values

In [ ]:
y_race

In [ ]:
rh_df['race_id']

In [ ]:
rh_df['horse_place'].dtype

In [ ]:
import datetime as dt
import numpy as np

In [ ]:
pd.api.types.is_datetime64tz_dtype(rh_df.dtypes["race_datetime"])

In [ ]:
rh_df.dtypes["horse_id"] == int

In [ ]:
np.datetime64()

In [ ]:
rh_df.dtypes["race_datetime"]

In [ ]:
pd.api.types.is_datetime64tz_dtype()

In [ ]:
from utils.import_data import *

In [ ]:
df['horse_place']=df['horse_place'].astype("Int64")

In [ ]:
df['horse_place']

In [ ]:
df=pd.read_parquet(os.path.join(DATA_DIR, "unibet_data_with_features.parquet"))

In [ ]:
df['horse_place']=df['horse_place'].astype(float)

In [ ]:
df['horse_place']

In [ ]:
df.to_parquet(os.path.join(DATA_DIR, "unibet_data_with_features.parquet"))

In [14]:
import os
import re
from typing import Optional

import numpy as np
from xgboost import XGBClassifier
from xgboost.core import XGBoostError

from constants import SAVED_MODELS_DIR, Sources
from winning_horse_models import AbstractWinningModel, FlattenMixin
from winning_horse_models import ModelNotCreatedOnceError
from sklearn.exceptions import NotFittedError
self=xgboost_winning_model
prefix="48_col_"
if self.__class__.__name__ not in os.listdir(SAVED_MODELS_DIR):
    os.mkdir(os.path.join(SAVED_MODELS_DIR, self.__class__.__name__))
for n_horse, n_horse_model in self.n_horses_models.items():
    print(n_horse)
    try:
        n_horse_model.save_model(
            fname=os.path.join(
                SAVED_MODELS_DIR,
                self.__class__.__name__,
                f"{prefix}{self.__class__.__name__}_{n_horse}.pickle",
            )
        )
    except XGBoostError as e:
        print(f"Could not save model for {n_horse} horses: {e}")
    except NotFittedError as e:
        print(f"Could not save model for {n_horse} horses: {e}")

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
26
Could not save model for 26 horses: need to call fit or load_model beforehand
27
Could not save model for 27 horses: need to call fit or load_model beforehand
28
29
Could not save model for 29 horses: need to call fit or load_model beforehand
30
31
Could not save model for 31 horses: need to call fit or load_model beforehand
33
Could not save model for 33 horses: need to call fit or load_model beforehand
35
Could not save model for 35 horses: need to call fit or load_model beforehand
36
Could not save model for 36 horses: need to call fit or load_model beforehand
40
43
Could not save model for 43 horses: need to call fit or load_model beforehand


In [ ]:
memory.clear()